<a href="https://colab.research.google.com/github/tuandung222/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [4]:
!pip install dotenv trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Get HF_TOKEN from environment variables
hf_token = os.environ.get("HF_TOKEN")

# Login to Hugging Face with the token
if hf_token:
    login(token=hf_token)
else:
    print("HF_TOKEN not found in environment variables. Please login manually:")
    login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

HF_TOKEN not found in environment variables. Please login manually:


In [5]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [6]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [7]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

train_dataset = ds["train"]
test_dataset = ds["test"]

README.md:   0%|          | 0.00/9.72k [00:00<?, ?B/s]

data/everyday-conversations/train-00000-(…):   0%|          | 0.00/946k [00:00<?, ?B/s]

data/everyday-conversations/test-00000-o(…):   0%|          | 0.00/52.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/119 [00:00<?, ? examples/s]

In [8]:
from datasets import load_dataset




def process_dataset(sample):
    # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template


    # Extract the messages from the sample
    messages = sample['messages']

    # Apply the chat template to create a formatted conversation
    formatted_conversation = tokenizer.apply_chat_template(
        messages,
        tokenize=False  # We want the text, not tokens
    )

    # Add the formatted conversation to the sample
    sample['formatted_conversation'] = formatted_conversation

    return sample


# ds = ds.map(process_dataset)

In [9]:
def formatting_func(example):
    messages = example['messages']
    # Apply the chat template to create a formatted conversation
    formatted_conversation = tokenizer.apply_chat_template(
        messages,
        tokenize=False  # We want the text, not tokens
    )
    # return text
    return formatted_conversation

In [29]:
formatting_func(train_dataset[2])

"<|im_start|>user\nHi<|im_end|>\n<|im_start|>assistant\nHello! How can I help you today?<|im_end|>\n<|im_start|>user\nI'm going to the mall to do some window shopping. What's the point of window shopping if I'm not going to buy anything?<|im_end|>\n<|im_start|>assistant\nWindow shopping can be a fun way to browse and get inspiration for future purchases, or to simply enjoy looking at products and displays without feeling pressured to buy.<|im_end|>\n<|im_start|>user\nThat makes sense. What are some basic rules I should follow while window shopping to be polite to the store staff?<|im_end|>\n<|im_start|>assistant\nSome basic rules include not touching or handling merchandise excessively, not blocking store entrances or aisles, and being respectful of staff and other customers.<|im_end|>\n<|im_start|>user\nAlright, I'll keep those in mind. Is it okay to take pictures of store displays or products?<|im_end|>\n<|im_start|>assistant\nYes, it's usually okay to take pictures of store displays

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [13]:
SFTConfig??

In [14]:
SFTTrainer??

In [15]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [16]:
!nvidia-smi

Thu May 15 02:36:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P0             30W /   70W |     670MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [17]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=2000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=2,  # Set according to your GPU memory capacity
    learning_rate=7e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=100,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    max_seq_length=512,  # Maximum sequence length for training
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    # tokenizer=tokenizer,
    processing_class=tokenizer,
    eval_dataset=ds["test"],
    formatting_func=formatting_func,
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

Applying formatting function to train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2260 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/119 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [18]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dung-vo20csehcmut (Private_1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
100,1.081000,1.129727
200,1.139300,1.106642
300,1.056800,1.077282
400,1.027000,1.063600
500,0.985900,1.051769
600,1.017000,1.037335
700,0.969100,1.036172
800,0.984800,1.028705
900,1.001900,1.018434
1000,1.070400,1.002644


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
trainer.push_to_hub(tags=finetune_tags)

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

events.out.tfevents.1747276625.486b5acac7f3.253.0:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tuandunghcmut/SmolLM2-FT-MyDataset/commit/714b3ac414921177eb075d71f664cdd46a776b64', commit_message='End of training', commit_description='', oid='714b3ac414921177eb075d71f664cdd46a776b64', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tuandunghcmut/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='tuandunghcmut/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [20]:
test_dataset[-1]

{'full_topic': 'Music/Musical instruments/Unusual instruments',
 'messages': [{'content': 'Hi there', 'role': 'user'},
  {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
  {'content': "I'm interested in learning about unusual musical instruments. Can you tell me about one?",
   'role': 'user'},
  {'content': "One unusual instrument is the Theremin. It's an electronic instrument played without physical contact, using hand movements to control pitch and volume.",
   'role': 'assistant'},
  {'content': "That sounds interesting. What's another unusual instrument?",
   'role': 'user'},
  {'content': "Another unusual instrument is the Hang Drum. It's a modern, hand-played drum that produces a rich, resonant sound when played with the fingers.",
   'role': 'assistant'},
  {'content': "I'd love to hear what it sounds like.", 'role': 'user'}]}

In [31]:
tokenizer.apply_chat_template??

In [36]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
# prompt = "Write a haiku about programming"
prompt = "What is the capital of France?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(formatted_prompt)
# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

outputs = model.generate(**inputs, max_new_tokens=512)

<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant



In [37]:
outputs[0]

tensor([   1, 4093,  198, 1780,  314,  260, 3575,  282, 4649,   47,    2,  198,
           1,  520, 9531,  198,  504, 3575,  282, 4649,  314, 7042,   30,    2],
       device='cuda:0')

In [38]:
formatted_prompt

'<|im_start|>user\nWhat is the capital of France?<|im_end|>\n<|im_start|>assistant\n'

In [39]:
# decoding the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("After training:")
print(decoded_output)

After training:
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
The capital of France is Paris.<|im_end|>


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.